In [1]:
import sys
sys.path.append("./reference")
import torch as t
from reference.glide_text2im.download import load_checkpoint
from reference.glide_text2im.model_creation import (
    create_model_and_diffusion,
    model_and_diffusion_defaults,
)

device = t.device('cpu')
options = model_and_diffusion_defaults()
options['timestep_respacing'] = '100'
ref_model, diffusion = create_model_and_diffusion(**options)
ref_model.load_state_dict(load_checkpoint('base', device))


<All keys matched successfully>

In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
from unet import Text2Im
from reference.glide_text2im.tokenizer.bpe import get_encoder

In [4]:
tokenizer = get_encoder()
model = Text2Im(tokenizer)
model

Text2Im(
  (time_embed): Sequential(
    (0): Linear(in_features=192, out_features=768, bias=True)
    (1): SiLU()
    (2): Linear(in_features=768, out_features=768, bias=True)
  )
  (in_layers): Sequential(
    (0): Conv2d(3, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ResidualBlock(
      (in_layers): Sequential(
        (0): GroupNorm(32, 192, eps=1e-05, affine=True)
        (1): SiLU()
        (2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (emb_layers): Sequential(
        (0): SiLU()
        (1): Linear(in_features=768, out_features=384, bias=True)
      )
      (group_norm): GroupNorm(32, 192, eps=1e-05, affine=True)
      (out_layers): Sequential(
        (0): SiLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (2): ResidualBlock(
      (in_layers): Sequential(
        (0): GroupNorm(32, 192, eps=1e-05, affine=True)
       

In [5]:
print(len(model.state_dict().keys()), len(ref_model.state_dict().keys()))
list(zip(model.state_dict().keys(), ref_model.state_dict().keys()))

783 783


[('positional_embedding', 'positional_embedding'),
 ('padding_embedding', 'padding_embedding'),
 ('time_embed.0.weight', 'time_embed.0.weight'),
 ('time_embed.0.bias', 'time_embed.0.bias'),
 ('time_embed.2.weight', 'time_embed.2.weight'),
 ('time_embed.2.bias', 'time_embed.2.bias'),
 ('in_layers.0.weight', 'input_blocks.0.0.weight'),
 ('in_layers.0.bias', 'input_blocks.0.0.bias'),
 ('in_layers.1.in_layers.0.weight', 'input_blocks.1.0.in_layers.0.weight'),
 ('in_layers.1.in_layers.0.bias', 'input_blocks.1.0.in_layers.0.bias'),
 ('in_layers.1.in_layers.2.weight', 'input_blocks.1.0.in_layers.2.weight'),
 ('in_layers.1.in_layers.2.bias', 'input_blocks.1.0.in_layers.2.bias'),
 ('in_layers.1.emb_layers.1.weight', 'input_blocks.1.0.emb_layers.1.weight'),
 ('in_layers.1.emb_layers.1.bias', 'input_blocks.1.0.emb_layers.1.bias'),
 ('in_layers.1.group_norm.weight', 'input_blocks.1.0.out_layers.0.weight'),
 ('in_layers.1.group_norm.bias', 'input_blocks.1.0.out_layers.0.bias'),
 ('in_layers.1.out_l

In [6]:
state = ref_model.state_dict()
mapped = {ours: state[theirs] for ours, theirs in zip(model.state_dict().keys(), ref_model.state_dict().keys())}

model.load_state_dict(mapped)

<All keys matched successfully>